In [1]:
import numpy as np
import pybullet as p
from multiprocessing import Manager, Value
from stable_baselines3.common.vec_env import (
    DummyVecEnv,
    SubprocVecEnv,
    VecNormalize,
    VecMonitor,
    VecFrameStack
)

import gymnasium as gym
from gym_pybullet_drones.utils.enums import DroneModel, Physics
from race_rl.nn import DronePolicy
from race_rl.env import RaceAviary, DeployType, DecreaseOmegaCoef



pybullet build time: Jun  5 2023 17:14:31


In [2]:
def make_env(track_path, gui, num, start_pos):
    for i in range(num): 
        env=RaceAviary(
        init_segment=i,
        start_dict=start_pos,
        drone_model=DroneModel('cf2x'),
        # TODO - change the intialil_xyzs
        physics=Physics('pyb'),
        pyb_freq=240,
        ctrl_freq=60,
        gui=gui,
        record=False,
        gates_lookup=2,
        track_path=track_path,
        world_box=[100,100,5],
        user_debug_gui=False, 
        coef_gate_filed=0,
        coef_omega=0,
        deploy_type=DeployType.TRAINING,
        max_distance_segmnet=10,
        )
        yield env

In [12]:
manager = Manager()
start_pos = manager.dict()
track_path = "/home/michunie/projects/magisterka/agh-drone-racing/assets/tracks/thesis-tracks/straight_track.csv"

observations = []
for env in make_env(track_path, False, 10, start_pos):
    env.reset()
    for _ in range(1000):
        while True:
            a = env.action_space.sample()
            observation, reward, terminated, truncated, info = env.step(a)
            observations.append(observation)
            done = truncated or terminated
            if done:
                env.reset()
                break
            
    
    del env

[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 0.000000, km 0.000000,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 0.000000, km 0.000000,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 0.000000, km 0.00

In [14]:
observations = np.array(observations)

In [15]:
observations.shape

(498520, 23)

In [22]:
np.std(observations, axis=0)+0.00000001

array([1.19471958e+00, 1.20550044e+00, 1.94390471e+00, 6.13552788e-01,
       5.32095437e-01, 5.39820782e-01, 5.30323524e-01, 6.14479642e-01,
       5.39636304e-01, 5.41564262e-01, 5.37897584e-01, 6.09331056e-01,
       1.21795905e+01, 1.23309015e+01, 1.08868059e+01, 1.15803975e+00,
       1.56313058e+00, 6.39920770e-01, 8.78764674e-01, 1.22554759e+00,
       1.24029474e-01, 3.43897864e-02, 1.00000000e-08])

In [20]:
np.mean(observations, axis=0)

array([ 4.91098773e-03,  3.49276799e-03, -2.36853503e+00,  2.21211360e-01,
       -8.77447053e-03,  3.08580949e-03,  9.11177272e-03,  2.23334406e-01,
        1.77231613e-03,  7.93421836e-04,  4.36446481e-04,  2.14683993e-01,
       -1.45240204e-02,  7.25515498e-02, -1.26163573e-01,  3.46472223e+00,
        3.82323314e-03,  1.57116349e+00,  1.78643652e+00,  4.44291603e+00,
        3.76638827e-02,  1.57903861e+00,  0.00000000e+00])

In [23]:
    std = np.std(observations, axis=0)
    std[-1] = 1

In [24]:
std

array([ 1.19471957,  1.20550043,  1.9439047 ,  0.61355278,  0.53209543,
        0.53982077,  0.53032351,  0.61447963,  0.53963629,  0.54156425,
        0.53789757,  0.60933105, 12.17959052, 12.33090145, 10.88680584,
        1.15803974,  1.56313057,  0.63992076,  0.87876466,  1.22554758,
        0.12402946,  0.03438978,  1.        ])

In [26]:
import pickle

with open("norm_straight.pkl", "wb") as f:
    std = np.std(observations, axis=0)
    std[-1] = 1
    obj = (np.mean(observations, axis=0), std)
    pickle.dump(obj, f)